Jetsim : Build Model
===

### **Step 4.2** from README.md

---

INITIAL SETUP

Right click and open 'this' notebook **JESTIM_interactive_regression.ipynb** in Google colab.

Open 'Edit/Notebook settings' toolbar in Google Colab and select **GPU**

If you get any runtime errors at anytime, go to 'Runtime' toolbar and click 'Restart Runtime'. Rerun all cells.

---


In [ ]:
#initialize google colab
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
print("DONE")


Mounted at /content/gdrive
DONE


Select 'jestim-google-colab' dirctory and unzip images
===

In [ ]:
%cd /content/gdrive/MyDrive/jetsim_googlecolab/SIM_road_following_A
!unzip m1_images.zip
%cd ..

Task - initialize data set
===

### Place all name-converted images into 'SIM_road_following_A/apex' folder before running this cell.

In [ ]:
import torchvision.transforms as transforms
from xy_dataset import XYDataset

#TASK = 'road_following'
TASK = 'SIM_road_following'

CATEGORIES = ['m1_images']

DATASETS = ['A']

TRANSFORMS = transforms.Compose([
    transforms.ColorJitter(0.2, 0.2, 0.2, 0.2),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

datasets = {}
for name in DATASETS:
    datasets[name] = XYDataset(TASK + '_' + name, CATEGORIES, TRANSFORMS, random_hflip=True)

print("DONE")

DONE


### Data Collection

In [ ]:
import cv2
import ipywidgets
import traitlets
from IPython.display import display

# initialize active dataset
dataset = datasets[DATASETS[0]]

dataset_widget = ipywidgets.Dropdown(options=DATASETS, description='dataset')
category_widget = ipywidgets.Dropdown(options=dataset.categories, description='category')
count_widget = ipywidgets.IntText(description='count')

# manually update counts at initialization
count_widget.value = dataset.get_count(category_widget.value)

# sets the active dataset
def set_dataset(change):
    global dataset
    dataset = datasets[change['new']]
    count_widget.value = dataset.get_count(category_widget.value)
dataset_widget.observe(set_dataset, names='value')

# update counts when we select a new category
def update_counts(change):
    count_widget.value = dataset.get_count(change['new'])
category_widget.observe(update_counts, names='value')


def save_snapshot(_, content, msg):
    if content['event'] == 'click':
        data = content['eventData']
        x = data['offsetX']
        y = data['offsetY']
        
        # save to disk
        dataset.save_entry(category_widget.value, camera.value, x, y)
        
        # display saved snapshot
        snapshot = camera.value.copy()
        snapshot = cv2.circle(snapshot, (x, y), 8, (0, 255, 0), 3)
        snapshot_widget.value = bgr8_to_jpeg(snapshot)
        count_widget.value = dataset.get_count(category_widget.value)
        
# camera_widget.on_msg(save_snapshot)

data_collection_widget = ipywidgets.VBox([
    dataset_widget,
    category_widget,
    count_widget
])

# ipywidgets.HBox([camera_widget, snapshot_widget]),
# display(data_collection_widget)

# Evalutate stuff, not needed for traing but need for code competion. 
import threading
import time
from utils import preprocess
import torch.nn.functional as F

state_widget = ipywidgets.ToggleButtons(options=['stop', 'live'], description='state', value='stop')

def live(state_widget, model, camera, prediction_widget):
    global dataset
            
def start_live(change):
  pass

print("DONE")

RuntimeError: ignored

Setup empty model
===

In [ ]:
import torch
import torchvision

device = torch.device('cuda')
output_dim = 2 * len(dataset.categories)  # x, y coordinate for each category

# ALEXNET
# model = torchvision.models.alexnet(pretrained=True)
# model.classifier[-1] = torch.nn.Linear(4096, output_dim)

# SQUEEZENET 
# model = torchvision.models.squeezenet1_1(pretrained=True)
# model.classifier[1] = torch.nn.Conv2d(512, output_dim, kernel_size=1)
# model.num_classes = len(dataset.categories)

# RESNET 18
model = torchvision.models.resnet18(pretrained=True)
model.fc = torch.nn.Linear(512, output_dim)

# RESNET 34
# model = torchvision.models.resnet34(pretrained=True)
# model.fc = torch.nn.Linear(512, output_dim)

# DENSENET 121
# model = torchvision.models.densenet121(pretrained=True)
# model.classifier = torch.nn.Linear(model.num_features, output_dim)

model = model.to(device)

model_save_button = ipywidgets.Button(description='save model')
model_load_button = ipywidgets.Button(description='load model')
model_path_widget = ipywidgets.Text(description='model path', value='NAME.pth')

def load_model(c):
    model.load_state_dict(torch.load(model_path_widget.value))
model_load_button.on_click(load_model)
    
def save_model(c):
    torch.save(model.state_dict(), model_path_widget.value)
model_save_button.on_click(save_model)

model_widget = ipywidgets.VBox([
    model_path_widget,
    ipywidgets.HBox([model_load_button, model_save_button])
])


display(model_widget)

rename model 'NAME.pth' and click SAVE. The click LOAD in the widget above.
===

Setup training parameters
===

In [ ]:
BATCH_SIZE = 8

optimizer = torch.optim.Adam(model.parameters())
#optimizer = torch.optim.SGD(model.parameters(), lr=1e-3, momentum=0.9)

epochs_widget = ipywidgets.IntText(description='epochs', value=10)
eval_button = ipywidgets.Button(description='evaluate')
train_button = ipywidgets.Button(description='train')
loss_widget = ipywidgets.FloatText(description='loss')
progress_widget = ipywidgets.FloatProgress(min=0.0, max=1.0, description='progress')

def train_eval(is_training):
    global BATCH_SIZE, LEARNING_RATE, MOMENTUM, model, dataset, optimizer, eval_button, train_button, accuracy_widget, loss_widget, progress_widget, state_widget
    
    try:
        train_loader = torch.utils.data.DataLoader(
            dataset,
            batch_size=BATCH_SIZE,
            shuffle=True
        )

        state_widget.value = 'stop'
        train_button.disabled = True
        eval_button.disabled = True
        time.sleep(1)

        if is_training:
            model = model.train()
        else:
            model = model.eval()

        while epochs_widget.value > 0:
            i = 0
            sum_loss = 0.0
            error_count = 0.0
            for images, category_idx, xy in iter(train_loader):
                # send data to device
                images = images.to(device)
                xy = xy.to(device)

                if is_training:
                    # zero gradients of parameters
                    optimizer.zero_grad()

                # execute model to get outputs
                outputs = model(images)

                # compute MSE loss over x, y coordinates for associated categories
                loss = 0.0
                for batch_idx, cat_idx in enumerate(list(category_idx.flatten())):
                    loss += torch.mean((outputs[batch_idx][2 * cat_idx:2 * cat_idx+2] - xy[batch_idx])**2)
                loss /= len(category_idx)

                if is_training:
                    # run backpropogation to accumulate gradients
                    loss.backward()

                    # step optimizer to adjust parameters
                    optimizer.step()

                # increment progress
                count = len(category_idx.flatten())
                i += count
                sum_loss += float(loss)
                progress_widget.value = i / len(dataset)
                loss_widget.value = sum_loss / i
                
            if is_training:
                epochs_widget.value = epochs_widget.value - 1
            else:
                break
    except e:
        pass
    model = model.eval()

    train_button.disabled = False
    eval_button.disabled = False
    state_widget.value = 'live'
    
train_button.on_click(lambda c: train_eval(is_training=True))
eval_button.on_click(lambda c: train_eval(is_training=False))
    
train_eval_widget = ipywidgets.VBox([
    epochs_widget,
    progress_widget,
    loss_widget,
    ipywidgets.HBox([train_button, eval_button])
])

#display(train_eval_widget)

print("DONE")

In [ ]:
all_widget = ipywidgets.VBox([
    ipywidgets.HBox([data_collection_widget]), train_eval_widget,
    model_widget
])

display(all_widget)

Epoch and training tips
===

---

- Epochs = 10.
- Click 'train' button.
- Watch blue progress bar, <5 minutes.
- Click 'save model' button when training is finished.

---



### Create the **models** folder to store all your models.

In [ ]:
import os
dir = "models"
try:
    os.makedirs(dir, exist_ok = False)
    print("Directory '%s' created successfully" % dir)
except OSError as error:
    print("Directory '%s' already exists" % dir)

### Move your 'model.pth' to **models** folder inside Google drive **manually**. 

### Close this notebook and proceed to **step 5** in README.md.

### Follow **JETSIM_road_following.ipynb** notbook to test model. 